# 행렬 분해를 이용한 추천서비스(Matrix Factorization)_특이값 분해

In [3]:
import pandas as pd

#csv파일 dataframe으로 변환하고 읽기
review_data_2 = pd.read_csv('./yogiyo/order_db_v2.csv', encoding='ansi')
                          #encoding='utf-8')
#불필요한 데이터 drop하기
review_data_2.drop('Unnamed: 0', axis=1, inplace=True)

review_data_2#.head(5)

,user_id,order_date,order_item,order_no,point,comment
0,lo,2020-11-23,궁중식 순살찜닭,9181,5,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요
1,ni,2020-11-23,허니갈릭 닭가슴살 시저샐러드,9180,4,무난무난 쉐플리 ㅇㅇ
2,qk,2020-11-23,궁중식 순살찜닭,9179,3,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥
3,dk,2020-11-23,두툼한 통등심 가츠동,9178,5,빠른 배달 감사합니다.
4,ji,2020-11-23,로스트 치킨과 매콤한 크림리조또,9177,5,맛있어요 ㅋㅋㅋㅋ 넘넘만족해용
...,...,...,...,...,...,...
15290,mo,2017-11-03,허니갈릭 닭가슴살 시저샐러드,2,5,건강하게 한끼 먹어서 좋았습니다. 샐러드 양도많고 퀄리티도 생각했던거 보다 좋네요....
15291,mo,2017-11-03,코카콜라 제로,2,5,건강하게 한끼 먹어서 좋았습니다. 샐러드 양도많고 퀄리티도 생각했던거 보다 좋네요....
15292,b9,2017-11-02,모둠나물 제육비빔밥,1,5,일하면서 먹어서 사진을 이쁘게 찍진 못했네요 ㅜㅜ. 일단 맛은 최고! 배달도 친절!...
15293,b9,2017-11-02,순살 새우치킨 콤보 (양념),1,5,일하면서 먹어서 사진을 이쁘게 찍진 못했네요 ㅜㅜ. 일단 맛은 최고! 배달도 친절!...


In [4]:
import pandas as pd

#csv파일 dataframe으로 변환하고 읽기
menu_data = pd.read_csv('./yogiyo/Menu_DB.csv', encoding='ansi')
                          #encoding='utf-8')

menu_data.head(10)

,menu_name,menu_categori,price,menu_exp
0,씨푸드 나시고랭,메인,10000,큼지막한 새우와 다양한 해산물이 쏙쏙! 이국적인 맛이 고스란히 담긴 인도네시아풍 볶음밥
1,그라나파다노 김치볶음밥,메인,10000,트러플향 가득! 르꼬르동블루 출신 원영호 셰프의 레시피로 만든 프리미엄 김치볶음밥
2,비빔명란 가라아케동,메인,10000,촉촉한 가라아게와 톡톡 터지는 명란이 더해진 근사한 별미 (명란을 잘 섞어드세요)
3,매콤대패삼겹덮밥,메인,11000,고소한 대패삼겹살에 특제 양념의 매콤달콤함을 더한 셰플리키친 시그니처 메뉴
4,모둠나물 제육비빔밥,메인,11000,매콤달콤한 제육불고기와 여섯 종류의 모둠나물이 조화롭게 어우러진 든든한 한 끼
5,소불고기 비빔밥,메인,11000,전 세계인이 사랑하는 한식의 대표주자 비빔밥과 불고기의 환상적 만남
6,궁중식 순살찜닭,메인,11000,면역력에 좋은 귀한재료가 아낌없이 듬뿍! 임금님도 반할 만한 궁중식 순살찜닭
7,두툼한 통등심 가츠동,메인,10000,육즙 가득 두툼한 생등심이 통으로! 남녀노소가 사랑하는 셰플리의 스터디셀러
8,대패삼겹덮밥,메인,11000,미쉐린 등재 레스토랑 '민스키친'의 김민지 셰프가 선보이는 든든한 한 끼
9,로스트 치킨과 매콤한 크림리조또,메인,11000,허브향 닭다리살이 통으로! 청양고추로 느끼함을 잡은 고급스러운 크림 리조또


In [5]:
#menu_data, review_data_2 merge 시키기
merge_data = pd.merge(review_data_2, menu_data, left_on='order_item', 
                      right_on='menu_name', how='outer')
merge_data.drop(['menu_name','menu_exp'], axis=1, inplace=True)
merge_data#.head(35)

,user_id,order_date,order_item,order_no,point,comment,menu_categori,price
0,lo,2020-11-23,궁중식 순살찜닭,9181,5,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요,메인,11000
1,qk,2020-11-23,궁중식 순살찜닭,9179,3,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥,메인,11000
2,ji,2020-11-23,궁중식 순살찜닭,9175,5,참 맛있고 배달 빨라요,메인,11000
3,co,2020-11-23,궁중식 순살찜닭,9171,5,잘먹었습니다 고기냄새도안나고 아주 좋아요,메인,11000
4,ja,2020-11-23,궁중식 순살찜닭,9163,3,배달원은 무뚝뚝 음식만 띡 주고가서 약간 불친절 음식은 뭐 식당에서 먹는 찜닭이랑 ...,메인,11000
...,...,...,...,...,...,...,...,...
15290,rh,2018-02-12,페리에 레몬탄산수,315,5,배달음식에서 맛보기 드문 훌륭한 맛입이다. 담백하게 맛있어요. 야채를 먹지않는 아들...,음료,2500
15291,ma,2018-02-07,페리에 레몬탄산수,294,5,매번 주문해서 먹을때마다 만족합니다~^^,음료,2500
15292,jg,2018-01-20,페리에 레몬탄산수,211,4,굳 맛있어용~~~ㅋㅋ,음료,2500
15293,gg,2017-12-18,페리에 레몬탄산수,119,5,궁중 비빔밥 맛있어요 양도 많고 제가 딱 원하던맛!! 자주 시켜먹을듯해요,음료,2500


In [6]:
#column 순서 변경

#'comment'를 제외한 모든 변수들 붙인다
cols_to_move = ['comment']
cols = [col for col in merge_data if col not in cols_to_move] + cols_to_move
merge_data[cols]

,user_id,order_date,order_item,order_no,point,menu_categori,price,comment
0,lo,2020-11-23,궁중식 순살찜닭,9181,5,메인,11000,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요
1,qk,2020-11-23,궁중식 순살찜닭,9179,3,메인,11000,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥
2,ji,2020-11-23,궁중식 순살찜닭,9175,5,메인,11000,참 맛있고 배달 빨라요
3,co,2020-11-23,궁중식 순살찜닭,9171,5,메인,11000,잘먹었습니다 고기냄새도안나고 아주 좋아요
4,ja,2020-11-23,궁중식 순살찜닭,9163,3,메인,11000,배달원은 무뚝뚝 음식만 띡 주고가서 약간 불친절 음식은 뭐 식당에서 먹는 찜닭이랑 ...
...,...,...,...,...,...,...,...,...
15290,rh,2018-02-12,페리에 레몬탄산수,315,5,음료,2500,배달음식에서 맛보기 드문 훌륭한 맛입이다. 담백하게 맛있어요. 야채를 먹지않는 아들...
15291,ma,2018-02-07,페리에 레몬탄산수,294,5,음료,2500,매번 주문해서 먹을때마다 만족합니다~^^
15292,jg,2018-01-20,페리에 레몬탄산수,211,4,음료,2500,굳 맛있어용~~~ㅋㅋ
15293,gg,2017-12-18,페리에 레몬탄산수,119,5,음료,2500,궁중 비빔밥 맛있어요 양도 많고 제가 딱 원하던맛!! 자주 시켜먹을듯해요


In [10]:
#세분화 작업_메인
#delete_comment = merge_data
#delete_comment.drop('comment', axis=1, inplace=True)
group_data = merge_data[cols].groupby(['menu_categori'])

for key, group in group_data:
    print('key : ', key)
    print('number : ', len(group))
    print(group.head(10))
    print("\n")

key :  메인
number :  11401
  user_id  order_date order_item  order_no  point menu_categori  price  \
0      lo  2020-11-23   궁중식 순살찜닭      9181      5            메인  11000   
1      qk  2020-11-23   궁중식 순살찜닭      9179      3            메인  11000   
2      ji  2020-11-23   궁중식 순살찜닭      9175      5            메인  11000   
3      co  2020-11-23   궁중식 순살찜닭      9171      5            메인  11000   
4      ja  2020-11-23   궁중식 순살찜닭      9163      3            메인  11000   
5      9b  2020-11-23   궁중식 순살찜닭      9157      5            메인  11000   
6      bi  2020-11-22   궁중식 순살찜닭      9150      5            메인  11000   
7      wg  2020-11-22   궁중식 순살찜닭      9134      5            메인  11000   
8      ch  2020-11-22   궁중식 순살찜닭      9132      5            메인  11000   
9      ki  2020-11-21   궁중식 순살찜닭      9126      5            메인  11000   

                                             comment  
0                    배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요  
1                      뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 

In [13]:
#세분화 작업_메인
main_menu_group = group_data.get_group('메인')
main_menu_group#.head()
#print(main_menu_group.head())

,user_id,order_date,order_item,order_no,point,menu_categori,price,comment
0,lo,2020-11-23,궁중식 순살찜닭,9181,5,메인,11000,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요
1,qk,2020-11-23,궁중식 순살찜닭,9179,3,메인,11000,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥
2,ji,2020-11-23,궁중식 순살찜닭,9175,5,메인,11000,참 맛있고 배달 빨라요
3,co,2020-11-23,궁중식 순살찜닭,9171,5,메인,11000,잘먹었습니다 고기냄새도안나고 아주 좋아요
4,ja,2020-11-23,궁중식 순살찜닭,9163,3,메인,11000,배달원은 무뚝뚝 음식만 띡 주고가서 약간 불친절 음식은 뭐 식당에서 먹는 찜닭이랑 ...
...,...,...,...,...,...,...,...,...
14605,kh,2018-01-01,순살 새우치킨 콤보 (반반),158,5,메인,12000,맛은 좋아요 시간이 좀 걸려요
14606,sa,2017-12-23,순살 새우치킨 콤보 (반반),131,3,메인,12000,깔끔하고 정갈해서 맛있어요
14607,dm,2017-12-16,순살 새우치킨 콤보 (반반),113,1,메인,12000,"주문한지 1시간 반 넘었고, 배달안왔는데 배달 완료라 뜨네여"
14608,ye,2017-11-27,순살 새우치킨 콤보 (반반),47,5,메인,12000,첫숟가락을 떠보니 건강식이다라는 생각이나네요


In [15]:
type(group_data)

pandas.core.groupby.generic.DataFrameGroupBy

In [229]:
#세분화 작업_사이드
side_menu_group = group_data.get_group('사이드')
side_menu_group#.head()
#print(side_menu_group.head())

,user_id,order_date,order_item,order_no,point,menu_categori,price,comment
4416,ji,2020-11-23,계란말이,9177,5,사이드,2500,맛있어요 ㅋㅋㅋㅋ 넘넘만족해용
4417,ji,2020-11-23,계란말이,9161,5,사이드,2500,잘 먹었습니다! :)
4418,en,2020-11-22,계란말이,9144,5,사이드,2500,나시고랭을 좋아해서 쿠폰있는김에 주문해봤는데 포장도 음식도 넘 깔끔하고 양이 2인분...
4419,be,2020-11-22,계란말이,9142,5,사이드,2500,빠르게 잘 받았습니다
4420,jp,2020-11-19,계란말이,9092,5,사이드,2500,할인해서 시켰는데 맛있게 먹었습니다!
...,...,...,...,...,...,...,...,...
15008,tk,2017-11-13,명란마요밥,21,4,사이드,2700,샐러드가 너무짜요ㅜ 감자스프는 너무 맛있어요 그린빈이 왜없나여???원래있었는데 재료...
15009,gu,2017-11-13,명란마요밥,19,5,사이드,2700,맛있게 잘 먹었습니다 만족스럽네요
15010,je,2017-11-10,명란마요밥,14,5,사이드,2700,"사이드로 주문한 스프, 메인보다 좋았어요. 잘익은 감자 송송. 늦은 밤 먹었는데 속..."
15011,sc,2017-11-09,명란마요밥,10,5,사이드,2700,요즘 매일 시켜먹고 있어요. 고급진 느낌이 좋네요~


## 메인메뉴 추천

In [230]:
#사용자별 각 메뉴에대한 맛평가 데이터 만들기
user_menu_rating = main_menu_group.pivot_table(
    'point', 
    index = 'user_id', 
    columns = 'order_item').fillna(0)
user_menu_rating.tail()

order_item,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,명란마요 가라아게 콤보,모둠나물 제육비빔밥,비빔명란 가라아케동,소불고기 비빔밥,수원왕갈비 치밥,순살 새우치킨 콤보 (반반),순살 새우치킨 콤보 (소이),순살 새우치킨 콤보 (양념),씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,허니갈릭 닭가슴살 시저샐러드
user_id,,,,,,,,,,,,,,,,,,
zm,0.0,0.000000,0.000000,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
zn,4.0,0.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zo,4.0,4.000000,4.714286,4.0,4.0,0.0,4.0,0.0,0.0,5.0,4.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0
zx,5.0,0.000000,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
zz,5.0,4.857143,4.833333,4.0,4.5,0.0,0.0,0.0,4.0,5.0,5.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0


In [231]:
import numpy as np

#matrix는 위에서만든 pivot_table값을 numpy matrix로 만듬
matrix = user_menu_rating.values

#user_rating_avg는 사용자의 평균 맛평가
user_rating_avg = np.mean(matrix, axis = 1)

#matrix_user_avg : 사용자-평가에대해 사용자 평균 평점을 뺀것
matrix_user_avg = matrix - user_rating_avg.reshape(-1, 1)

matrix

array([[0.        , 0.        , 5.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 4.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [4.        , 4.        , 4.71428571, ..., 0.        , 0.        ,
        0.        ],
       [5.        , 0.        , 5.        , ..., 0.        , 0.        ,
        5.        ],
       [5.        , 4.85714286, 4.83333333, ..., 0.        , 0.        ,
        0.        ]])

In [232]:
#614명의 사용자가 평가한 점수의 평균데이터
pd.DataFrame(matrix_user_avg, columns=user_menu_rating.columns).head()
#pd.DataFrame(matrix_user_avg, index=user_menu_rating.index, columns=user_menu_rating.columns)

order_item,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,명란마요 가라아게 콤보,모둠나물 제육비빔밥,비빔명란 가라아케동,소불고기 비빔밥,수원왕갈비 치밥,순살 새우치킨 콤보 (반반),순살 새우치킨 콤보 (소이),순살 새우치킨 콤보 (양념),씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,허니갈릭 닭가슴살 시저샐러드
0,-1.111111,-1.111111,3.888889,3.888889,3.888889,-1.111111,-1.111111,-1.111111,-1.111111,-1.111111,-1.111111,-1.111111,-1.111111,-1.111111,3.888889,-1.111111,-1.111111,-1.111111
1,-0.277778,-0.277778,-0.277778,4.722222,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778
2,-0.944444,-0.944444,3.055556,-0.944444,3.055556,-0.944444,-0.944444,-0.944444,3.055556,-0.944444,4.055556,-0.944444,-0.944444,-0.944444,-0.944444,-0.944444,-0.944444,-0.944444
3,1.361111,3.361111,3.361111,3.361111,1.861111,-1.638889,-1.638889,-1.638889,-1.638889,-1.638889,1.361111,-1.638889,-1.638889,-1.638889,-1.638889,-1.638889,-1.638889,3.361111
4,-2.166667,2.833333,2.833333,2.833333,2.833333,-2.166667,-2.166667,1.833333,-2.166667,2.833333,-2.166667,-2.166667,2.833333,-2.166667,2.833333,-2.166667,-2.166667,-2.166667


In [233]:
from scipy.sparse.linalg import svds
#scipy에서제공하는 svd
#U행렬, sigma행렬, V전치행렬 반환
U, sigma, Vt = svds(matrix_user_avg, k=2)

print(U.shape)
print(sigma.shape)
print(Vt.shape)
#sigma 행렬은 0이 포함되지 않는 값으로만 되어있음

(614, 2)
(2,)
(2, 18)


In [234]:
#위에 sigma행렬을 대칭행렬로 변환
sigma = np.diag(sigma)
print(sigma.shape)
print(sigma[0])
print(sigma[1])

(2, 2)
[50.99536909  0.        ]
[ 0.         98.20019624]


In [235]:
#matrix_user_avg를 svd를 적용해 분해한 상태이므로 원복 필요
svd_user_predicted_rating = np.dot(np.dot(U, sigma), Vt) + user_rating_avg.reshape(-1,1)


In [236]:
svd_preds = pd.DataFrame(svd_user_predicted_rating, 
                         index=user_menu_rating.index, 
                         columns=user_menu_rating.columns)
svd_preds

order_item,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,명란마요 가라아게 콤보,모둠나물 제육비빔밥,비빔명란 가라아케동,소불고기 비빔밥,수원왕갈비 치밥,순살 새우치킨 콤보 (반반),순살 새우치킨 콤보 (소이),순살 새우치킨 콤보 (양념),씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,허니갈릭 닭가슴살 시저샐러드
user_id,,,,,,,,,,,,,,,,,,
0,0.806477,2.114634,3.615649,2.213171,2.668437,1.349392,0.438476,0.748819,0.624225,2.451660,0.914649,-0.116181,-0.207972,-0.323283,0.718875,-0.188839,-0.135603,2.307414
0_,1.309885,0.530058,0.691273,1.445521,0.173922,0.040416,-0.192467,0.261161,0.245399,0.357880,1.378580,-0.403473,-0.356778,-0.365562,-0.097545,-0.005828,-0.052053,0.039611
0f,1.628459,1.520491,2.214676,2.267055,1.454642,0.850938,0.325042,0.794164,0.734698,1.514932,1.722702,-0.048787,-0.047427,-0.097966,0.504924,0.231704,0.216086,1.213667
1,2.761877,3.054780,4.869885,4.441288,3.142365,1.558743,0.268074,1.232012,1.078013,3.170483,2.972625,-0.614740,-0.646530,-0.780927,0.697912,-0.134741,-0.143113,2.571994
10,1.229277,3.313901,5.144718,2.953259,4.210233,2.597029,1.557728,1.712925,1.562219,3.835704,1.330888,0.961669,0.819659,0.677099,1.865300,0.657602,0.750137,3.820651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zm,-0.422160,0.783779,1.302236,0.076377,1.315612,0.853317,0.649337,0.412150,0.371065,1.070928,-0.431564,0.581837,0.503793,0.461014,0.693088,0.224526,0.285474,1.269192
zn,2.576753,1.041173,1.206820,2.703790,0.343633,0.212973,-0.134063,0.667382,0.649288,0.698002,2.692876,-0.471695,-0.378593,-0.384745,0.015176,0.260381,0.171443,0.129405
zo,3.198659,3.077702,3.515204,3.600238,3.012307,2.632301,2.293097,2.612919,2.575304,3.062219,3.261336,2.048790,2.052899,2.021254,2.410191,2.248876,2.236046,2.854944


In [237]:
def recommend_menu(svd_preds, og_user_id, og_order_item, og_point, num_recommendations=5):
    
    #맛평가 평점이 높은 순으로 정렬
    sort_user_predictions = svd_preds.index.sort_values(ascending=False)
    
    #원본데이터에서 user_id에 해당하는 데이터 뽑기
    user_data = og_point[og_point.user_id == og_user_id]
    
    #원본데이터에서 사용자가 주문했던 음식은 제외한 데이터 추출
    recommendations = og_order_item[-og_order_item['order_item'].isin(user_data['order_item'])]  
    #사용자의 메뉴 평점이 높은 순으로 정렬된 데이터와 recommendations를 합친다
    recommendations = recommendations.merge(pd.DataFrame(sort_user_predictions).reset_index(), on = 'user_id')
    #컬럼이름을 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {'index' : 'Predictions'}).sort_values('Predictions', ascending=False)
    
    return user_data, recommendations

In [238]:
already_rated, predictions = recommend_menu(svd_preds, 'dk', main_menu_group, 
                                            main_menu_group, 10)
already_rated = already_rated.sort_values(by='order_no', ascending=False)
#predictions = predictions.sort_values(by='order_no', ascending=False)

In [239]:
already_rated

,user_id,order_date,order_item,order_no,point,menu_categori,price,comment
1635,dk,2020-11-23,두툼한 통등심 가츠동,9178,5,메인,10000,빠른 배달 감사합니다.
1644,dk,2020-11-22,두툼한 통등심 가츠동,9130,5,메인,10000,최고였습니다. 또 시킬께요!
5990,dk,2020-10-27,대패삼겹덮밥,8866,5,메인,11000,맛있어요!! 또 시켜먹을거같아요
11179,dk,2020-10-26,그라나파다노 김치볶음밥,8861,5,메인,10000,새로운 볶음밥에 감탄했어요~ 단백질양이 부족해보일땐 사이드로 추가주문도 할 수 있어...
11192,dk,2020-10-10,그라나파다노 김치볶음밥,8687,5,메인,10000,오랜만에 시켰는데 맛있네요
3464,dk,2020-09-07,로스트 치킨과 매콤한 크림리조또,8304,5,메인,11000,맛있는데 약간 텍스쳐가 구림
1771,dk,2020-09-07,두툼한 통등심 가츠동,8303,4,메인,10000,할인 받아서 맛있게 잘 먹었습니다~
1832,dk,2020-08-13,두툼한 통등심 가츠동,7949,1,메인,10000,서초점보다 질기고 냄새도 많이 났습니다. 잘 먹었습니다.
14466,dk,2020-03-24,어향가지 덮밥&멘보샤,6622,5,메인,16000,역시 맛이 잇어요오오오오오오오오오옹
10126,dk,2020-03-24,어향가지 덮밥,6622,5,메인,11000,역시 맛이 잇어요오오오오오오오오오옹


In [241]:
predictions.head(10)

,user_id,order_date,order_item,order_no,point,menu_categori,price,comment,Predictions
1497,0,2020-10-08,씨푸드 나시고랭,8677,5,메인,10000,맛있게 잘먹었습니다,613
1096,1,2019-08-18,허니갈릭 닭가슴살 시저샐러드,4069,5,메인,11000,맛있엇습니다~~~//~,610
1097,1,2019-07-19,허니갈릭 닭가슴살 시저샐러드,3623,5,메인,11000,양도 많고 맛있어요 !!,610
1098,1,2019-07-18,허니갈릭 닭가슴살 시저샐러드,3600,5,메인,11000,맛있게 잘먹었습니다 ㅎㅎ~~,610
1413,10,2020-11-16,씨푸드 나시고랭,9062,5,메인,10000,너무 맛나요 ㅋㅋ 특히 계란말이는 감동 ㅋ,609
1412,10,2020-11-09,순살 새우치킨 콤보 (소이),9009,5,메인,10000,맛있음 그러나 익스프레스인데 배달이 늦었음,609
1414,10,2019-11-21,모둠나물 제육비빔밥,5461,4,메인,11000,여수새꼬막 괜찮아요,609
1578,11,2020-08-20,순살 새우치킨 콤보 (반반),8010,4,메인,12000,감자튀김이 조금 더 바삭했으면 좋았겠지만 맛있네요,608
1577,11,2020-04-20,모둠나물 제육비빔밥,6804,5,메인,11000,양이 많네요 잘먹었습니다,608
1018,15,2019-09-24,허니갈릭 닭가슴살 시저샐러드,4745,5,메인,11000,요기요 덕분에 퀄리티 좋은거 먹어요,605
